# Variables and setting up a session 

In [2]:
import tensorflow as tf

x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')

f = x*x*y + y + 2


c:\users\yc\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
#Setting up a session
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)

result = sess.run(f)
print(result)
sess.close()    #always be closing

42


In [4]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
print(result)    

42


In [5]:
#Using tf.global_variables_initializer() to initialize all variables together
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)    #or init.run()
    result = f.eval()
print(result)

42


In [6]:
#With InteractiveSession()
sess = tf.InteractiveSession()
init.run()    #or sess.run(init)
result = f.eval()
#print(result)

#you have to close the session if you use InteractiveSession()
sess.close()

# Managing Graphs

In [7]:
#Any node you create is automatically added to the default graph
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()


True

In [8]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

print(x2.graph is graph)    #it is a graph
print(x2.graph is tf.get_default_graph)    #but not the  default graph
tf.reset_default_graph()

True
False


In [9]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x *3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


# Linear Regression with Tensorflow

In [10]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]    #added a column vector of m 1s

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

#In the evaluation phase, we will have to evaluate theta
with tf.Session() as sess:
    theta_value = theta.eval()

#This was the normal equation method
print(housing)    #what the data looks like

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
          37.88      , -122.23      ],
       [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
          37.86      , -122.22      ],
       [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
          39.43      , -121.32      ],
       [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
          39.37      , -121.24      ]]), 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]), 'feature_names': ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude'], 'DESCR': 'California housing dataset.\n\nThe original database is available from StatLib\n\n    http://lib.stat.cmu.edu/da

# Manually computing Gradients for GD
#### Batch Gradient Descent

In [11]:
from sklearn.preprocessing import StandardScaler
n_epochs = 1000    #times it will do forward and backward passes
learning_rate = 0.01

#We need to scale the Data for Gradient Descent
scaler = StandardScaler().fit(housing_data_plus_bias)
scaled_housing_data_plus_bias = scaler.transform(housing_data_plus_bias)

#Construction phase
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")    #tensor containing random values, given shape and range
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
#gradients = 2/m * tf.matmul(tf.transpose(X), error)    #-> Normal way
gradients = tf.gradients(mse, [theta])[0]              #-> Tensorflow way (efficient and accurate)
training_op = tf.assign(theta, theta - learning_rate * gradients)    #theta = theta - lr*gradients


#Execution Phase
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch: ", epoch, "MSE = ", mse.eval())
        sess.run(training_op)    #we construct the best theta here, eval later to find best
    best_theta = theta.eval()
print("Best theta: ", best_theta)

Epoch:  0 MSE =  8.166518
Epoch:  100 MSE =  4.900571
Epoch:  200 MSE =  4.837086
Epoch:  300 MSE =  4.8263326
Epoch:  400 MSE =  4.8203263
Epoch:  500 MSE =  4.8160243
Epoch:  600 MSE =  4.8128543
Epoch:  700 MSE =  4.810499
Epoch:  800 MSE =  4.808747
Epoch:  900 MSE =  4.8074436
Best theta:  [[ 0.08175325]
 [ 0.84664184]
 [ 0.13867585]
 [-0.26639062]
 [ 0.29264605]
 [ 0.0025235 ]
 [-0.0413745 ]
 [-0.7386512 ]
 [-0.7097736 ]]


## Using an Optimizer

In [12]:
#Add this in place of the gradients = [...] line

#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
#training_op = optimizer.minimize(mse)


In [13]:
#to feed data to algorithm, we make some modifications to the code:

#Construction phase
X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

batch_size = 100
n_batches = int(np.ceil(m / batch_size))

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  
    indices = np.random.randint(m, size=batch_size) 
    X_batch = scaled_housing_data_plus_bias[indices] 
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, y_batch 

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")    #tensor containing random values, given shape and range
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
#gradients = 2/m * tf.matmul(tf.transpose(X), error)    #-> Normal way
gradients = tf.gradients(mse, [theta])[0]              #-> Tensorflow way (efficient and accurate)
training_op = tf.assign(theta, theta - learning_rate * gradients)    #theta = theta - lr*gradients


#Execution Phase
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})    #placeholders are fed with this dict
        best_theta = theta.eval()
            
print("Best theta: ", best_theta)


Best theta:  [[-0.6091962 ]
 [ 0.8377844 ]
 [ 0.10645497]
 [-0.25947902]
 [ 0.29196444]
 [ 0.00181689]
 [ 0.2128084 ]
 [-0.89034677]
 [-0.85242176]]


# Saving and restoring models

- Create a Saver node at the end of construction phase

In [14]:
#Construction phase
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")    #tensor containing random values, given shape and range
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
#gradients = 2/m * tf.matmul(tf.transpose(X), error)    #-> Normal way
gradients = tf.gradients(mse, [theta])[0]              #-> Tensorflow way (efficient and accurate)
training_op = tf.assign(theta, theta - learning_rate * gradients)    #theta = theta - lr*gradients

#Saver:
saver = tf.train.Saver()

#Execution Phase
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch: ", epoch, "MSE = ", mse.eval())
        sess.run(training_op)    #we construct the best theta here, eval later to find best
    best_theta = theta.eval()
    save_path = saver.save(sess, "/tmp/ycModel.ckpt")
print("Best theta: ", best_theta)


Epoch:  0 MSE =  8.983538
Epoch:  100 MSE =  5.0793886
Epoch:  200 MSE =  4.9436193
Epoch:  300 MSE =  4.9025807
Epoch:  400 MSE =  4.8751645
Epoch:  500 MSE =  4.855452
Epoch:  600 MSE =  4.841187
Epoch:  700 MSE =  4.8308473
Epoch:  800 MSE =  4.823356
Epoch:  900 MSE =  4.8179173
Best theta:  [[-0.22724438]
 [ 0.8073278 ]
 [ 0.15336668]
 [-0.15084909]
 [ 0.17895629]
 [ 0.00837811]
 [-0.04177898]
 [-0.6684767 ]
 [-0.63302714]]


### To  restore

In [15]:
#Instead of using the init node during execution,
#Don't run
with tf.Session() as sess:
    saver.restore(sess, "tmp/ycModel.ckpt")


INFO:tensorflow:Restoring parameters from tmp/ycModel.ckpt


InvalidArgumentError: Unsuccessful TensorSliceReader constructor: Failed to get matching files on tmp/ycModel.ckpt: Not found: FindFirstFile failed for: tmp : The system cannot find the path specified.
; No such process
	 [[Node: save/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/RestoreV2/tensor_names, save/RestoreV2/shape_and_slices)]]

Caused by op 'save/RestoreV2', defined at:
  File "c:\users\yc\appdata\local\programs\python\python36\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\yc\appdata\local\programs\python\python36\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\yc\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\yc\appdata\local\programs\python\python36\lib\site-packages\traitlets\config\application.py"

# Visualizing with TensorBoard

- We need to use a different log directory everytime we run our program, else tf merges stats from two runs

In [16]:
tf.reset_default_graph()    #Clears the default graph stack and resets the global default graph.

from datetime import datetime  

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}".format(root_logdir, now)


In [17]:
#Construction phase
X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

batch_size = 100
n_batches = int(np.ceil(m / batch_size))

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")    #tensor containing random values, given shape and range
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
#gradients = 2/m * tf.matmul(tf.transpose(X), error)    #-> Normal way
gradients = tf.gradients(mse, [theta])[0]              #-> Tensorflow way (efficient and accurate)
training_op = tf.assign(theta, theta - learning_rate * gradients)    #theta = theta - lr*gradients

#Adding at end of construction for tensorboard:
mse_summary = tf.summary.scalar('MSE', mse)    #binary log string
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())


In [18]:
#Execution Phase
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            
            #Adding for tensorboard:
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
        
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})    #placeholders are fed with this dict
        best_theta = theta.eval()
            
print("Best theta: ", best_theta)
#close FileWriter at the end of program
file_writer.close()

Best theta:  [[-0.5685909 ]
 [ 0.8377844 ]
 [ 0.10645497]
 [-0.25947902]
 [ 0.29196444]
 [ 0.00181689]
 [ 0.2128084 ]
 [-0.89034677]
 [-0.85242176]]
